## Import Libraries

In [157]:

import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle

from skimage.util.shape import view_as_blocks
from skimage import io, transform

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D

import warnings
warnings.filterwarnings('ignore')

## Data Acquisition

In [158]:
train_size = 5000
test_size = 1000

DIR = "./dataset/"

dirs = ["train", "test"]

train = []
test = []

test_labels = []
train_labels = []
    
train_dir = os.path.join(DIR, "train")
test_dir = os.path.join(DIR, "test")

""" Extract FEN and filename """

for file in os.listdir(train_dir):

    base = os.path.basename(file)
    train_labels.append(os.path.splitext(base)[0])

    train.append(os.path.join(train_dir, file))

for file in os.listdir(test_dir):

    base = os.path.basename(file)
    test_labels.append(os.path.splitext(base)[0])

    test.append(os.path.join(test_dir, file))


# shuffle(train)
# shuffle(test)

train = train[:train_size]
test = test[:test_size]

train_labels = train_labels[:train_size]
test_labels = test_labels[:test_size]

In [159]:
assert len(train) == train_size and  len(test) == test_size

In [160]:
print(train_labels[0])
print(train_labels[1])
print(train_labels[2])

1b1B1b2-2pK2q1-4p1rB-7k-8-8-3B4-3rb3
1b1b1b2-3r4-1rK4b-R7-R2R1k2-2Bp4-2P5-2r5
1B1B1K2-3p1N2-6k1-R7-5P2-4q3-7R-1B6


## Exploratory Data Analysis

In [175]:
f, axs = plt.subplots(2,4, figsize=(240, 120))


for i in range(2):
    for j in range(4):
        
        r = np.random.randint(0, train_size - 1)
        
        axs[i][j].set_title(train_labels[r], fontsize=120, pad=30)
        axs[i][j].imshow(mpimg.imread(train[r]))
        axs[i][j].axis('off')
        
plt.savefig('boards.png')

## Feature Extraction

In [162]:
""" Convert FEN to one-hot encoded vectors and vice-versa """

pieces = 'prbnkqPRBNKQ'
numbers = '12345678'

def onehot(fen):

    onehot_encoding = []
    board = fen.split('-')

    for row in board:
        for sq in row:

            square = np.zeros(13)

            if sq in numbers:
                square[12] = 1

                for i in range(int(sq)):
                    onehot_encoding.append(square)
            
            else:
                square[pieces.index(sq)] = 1
                onehot_encoding.append(square)


    return np.array(onehot_encoding)

def fen(onehot):

    board = []

    for row in range(0, 64, 8):

        tmp = ""

        for sq in range(row, row+8, 1):

            n = np.argmax(onehot[sq])

            if n == 12:
                tmp += ' ' 
            else:
                tmp += pieces[n]

        for i in range(8, 0, -1):
            tmp = tmp.replace(' ' * i, str(i))

        board.append(tmp)

    
    fen = "-".join(board)

    return fen


assert train_labels[0] == fen(onehot(train_labels[0]))

In [163]:
def process_image(img):
    
    downsample_size = 200
    square_size = downsample_size // 8

    img_read = io.imread(img)
    img_read = transform.resize(img_read, (downsample_size, downsample_size), mode='constant')
    
    tiles = view_as_blocks(img_read, block_shape=(square_size, square_size, 3))
    tiles = tiles.squeeze(axis=2)

    return tiles.reshape(64, square_size, square_size, 3)

In [164]:
# df = pd.DataFrame({"position": train,
#                    "fen": train_labels})
# df["onehot"] = df["fen"].apply(lambda f : onehot(f))
# df["img"] = df["position"].apply(lambda p : process_image(p))

In [165]:
def train_generator(X_train, y_train):

    for i in range(train_size):

        y = onehot(y_train[i])
        x = process_image(X_train[i])

        yield x, y

def pred_generator(X_test):

    for i in range(test_size):
        yield process_image(X_test[i])


## Model Training

In [166]:
"""  Define the model """

model = Sequential()

model.add(Convolution2D(32, (3, 3), input_shape=(25, 25, 3)))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(13))
model.add(Activation('softmax'))

model.compile(
              loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [171]:
model.layers

In [167]:
""" Train the model """

model.fit_generator(train_generator(train, train_labels), steps_per_epoch=train_size)

5000/5000 [==============================] - 702s 140ms/step - loss: 0.0388 - accuracy: 0.9897


## Model Evaluation

In [168]:
""" Test the model """

y_pred = model.predict_generator(pred_generator(test), steps=test_size)
y_pred = y_pred.reshape(test_size, 64, 13)


In [169]:
assert y_pred.shape == (test_size, 64, 13)

In [170]:
y_fen = [fen(onehot) for onehot in y_pred]

print(classification_report(test_labels, y_fen))


                                               precision    recall  f1-score   support

               1B1B2K1-1B6-5N2-6k1-8-8-8-4nq2       1.00      1.00      1.00         1
               1B1B3R-8-1b2k3-8-2n5-1rK5-b7-8       1.00      1.00      1.00         1
          1B1B4-4R3-2R2p2-K7-8-3b3r-2Q4P-R2k4       1.00      1.00      1.00         1
         1B1BK3-7b-3Np3-4N3-2p1r2P-2r5-8-5k1b       1.00      1.00      1.00         1
                   1B1K3k-8-8-2r4R-2b5-8-7b-8       1.00      1.00      1.00         1
              1B1K4-1bb3Bb-4P3-5R2-R7-8-8-6k1       1.00      1.00      1.00         1
             1B1K4-1p5N-7p-1qp5-n1P5-8-6k1-b7       1.00      1.00      1.00         1
        1B1K4-2b5-8-8-R3R2P-5k2-N3R1r1-1r2RR2       1.00      1.00      1.00         1
              1B1K4-3B4-3p4-8-4k3-6p1-2B5-nq6       1.00      1.00      1.00         1
      1B1K4-3N1k1P-6Q1-3r1n2-8-3p1NNN-8-3b2rR       1.00      1.00      1.00         1
               1B1K4-3P4-6k1-1p6-6p1-8-7p-